In [ ]:
pip install impyute

In [ ]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from impyute.imputation.cs import mice
from sklearn.model_selection import train_test_split

In [ ]:
test = pd.read_csv('/content/drive/My Drive/data/kaggle/titanic/test.csv', encoding = 'cp949')
train = pd.read_csv('/content/drive/My Drive/data/kaggle/titanic/train.csv', encoding = 'cp949')


# 1.컬럼 확인

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
train.tail(5)

In [ ]:
test.isnull().sum()

#2.전처리

## 필요없는 컬럼 삭제

In [ ]:
train.drop("PassengerId", axis = 1, inplace = True)
test.drop("PassengerId", axis = 1, inplace = True)
train.drop("Name", axis = 1, inplace = True)
test.drop("Name", axis = 1, inplace = True)
train.drop("Ticket", axis = 1, inplace = True)
test.drop("Ticket", axis = 1, inplace = True)

In [ ]:
train.info()

## 편향성 확인

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = train['Age'], y = train['Survived'])
plt.ylabel('Survived', fontsize=13)
plt.xlabel('Age', fontsize=13)
plt.show()

In [ ]:
train[['Pclass']].boxplot()

In [ ]:
train[['Age']].boxplot()

In [ ]:
#Correlation map to see how features are correlated with SalePrice
import seaborn as sns

corrmat = train.corr()
plt.subplots(figsize=(12,9))
sns.heatmap(data = corrmat, annot=True, 
fmt = '.2f', linewidths=.5, cmap='Blues')

## 결측치 처리

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
# train["Embarked"] = train["Embarked"].fillna("None")

In [ ]:
# del train['Ticket']
# del test['Ticket']

In [ ]:
train.head()

In [ ]:
# test["Fare"] = test["Fare"].fillna("None")

## 라벨인코더

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_result = le.fit_transform(train['Embarked'])
test_result = le.fit_transform(test['Embarked'])

# print(result)
train['Embarked'] = train_result
test['Embarked'] = test_result

## 다중대체법

In [ ]:
train_no_Survived = train.drop(['Survived'], axis=1)


In [ ]:
total = pd.concat([train_no_Survived,test],axis=0,ignore_index=True)


In [ ]:
import missingno as msno
msno.matrix(total)


In [ ]:
dummy = pd.get_dummies(total)

In [ ]:
dummy.shape

In [ ]:
total_impute = pd.DataFrame(IterativeImputer(verbose=False).fit_transform(dummy))
total_impute

In [ ]:
msno.matrix(total_impute)


In [ ]:
total_cols = list(dummy.columns)
total_impute.columns = total_cols

In [ ]:
X_train = total_impute[:891]
X_test = total_impute[891:]

In [ ]:
y_train = train['Survived']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=777)


In [ ]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [ ]:
pd.options.display.float_format = '{:.5f}'.format


In [ ]:
Numeric_feature = [ col for col in train.columns if train[col].dtypes != "object"]
Numeric_feature

In [ ]:
y, X = dmatrices('Survived ~ Pclass+Age+SibSp+Parch+Fare', train, return_type = 'dataframe')

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns 
vif

In [ ]:
import statsmodels.api as sm;
import scipy.stats as stats
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
category_feature = [ col for col in train.columns if train[col].dtypes == "object"]
train_cor = train[category_feature]
train_cor

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score,explained_variance_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn import metrics

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X=X_train, y=y_train)
y_pred = model.predict(X_test) # 예측치
y_true = y_test # 정답
print('Root Mean Squared Error:',rmse(y_pred,y_test))

In [ ]:
from sklearn.svm import SVC 
model = SVC()  # kernel = 'rbf'  : default
model.fit(X=X_train, y=y_train)
y_pred = model.predict(X_test) # 예측치
y_true = y_test # 정답
print('Root Mean Squared Error:',rmse(y_pred,y_test))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier().fit(X=X_train, y=y_train)
model.predict(X_test)
y_pred = model.predict(X_test) # 예측치
print('Root Mean Squared Error:',rmse(y_pred,y_test))


In [ ]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier().fit(X=X_train, y=y_train)
model.predict(X_test)
y_pred = model.predict(X_test) # 예측치
print('Root Mean Squared Error:',rmse(y_pred,y_test))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier().fit(X_train, y_train)
model.predict(X_test)
y_pred = model.predict(X_test) # 예측치
print('Root Mean Squared Error:',rmse(y_pred,y_test))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(X_train, y_train)
model.predict(X_test)
y_pred = model.predict(X_test) # 예측치
print('Root Mean Squared Error:',rmse(y_pred,y_test))

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)
model.predict(X_test)
y_pred = model.predict(X_test) # 예측치
print('Root Mean Squared Error:',rmse(y_pred,y_test))